<a href="https://colab.research.google.com/github/Sujithra-dhayalan/MiniProject/blob/main/SepsiPre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [9]:
df_train = pd.read_csv('/content/Paitients_Files_Train.csv')
df_train.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,Sepssis
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive
1,ICU200011,1,85,66,29,0,26.6,0.351,31,0,Negative
2,ICU200012,8,183,64,0,0,23.3,0.672,32,1,Positive
3,ICU200013,1,89,66,23,94,28.1,0.167,21,1,Negative
4,ICU200014,0,137,40,35,168,43.1,2.288,33,1,Positive


In [10]:
df_train['prediction'] = df_train['Sepssis'].map({'Positive': 1, 'Negative': 0})
df_train.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,Sepssis,prediction
0,ICU200010,6,148,72,35,0,33.6,0.627,50,0,Positive,1
1,ICU200011,1,85,66,29,0,26.6,0.351,31,0,Negative,0
2,ICU200012,8,183,64,0,0,23.3,0.672,32,1,Positive,1
3,ICU200013,1,89,66,23,94,28.1,0.167,21,1,Negative,0
4,ICU200014,0,137,40,35,168,43.1,2.288,33,1,Positive,1


In [11]:
df_train = df_train.drop(['ID', 'Sepssis'], axis='columns')
df_train.head()

,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,prediction
0,6,148,72,35,0,33.6,0.627,50,0,1
1,1,85,66,29,0,26.6,0.351,31,0,0
2,8,183,64,0,0,23.3,0.672,32,1,1
3,1,89,66,23,94,28.1,0.167,21,1,0
4,0,137,40,35,168,43.1,2.288,33,1,1


In [12]:
x = df_train.drop('prediction', axis='columns')
y = df_train['prediction']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)


In [15]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))


In [16]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [26]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=100000, class_weight=class_weight_dict),
    'Random Forest': RandomForestClassifier(class_weight=class_weight_dict),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(scale_pos_weight=class_weight_dict[0] / class_weight_dict[1])  # For handling class imbalance
}

In [27]:
model_names = []
model_avg_score = []

for model_name, model in models.items():
    scores = cross_val_score(model, x_train_scaled, y_train, scoring='roc_auc', cv=5)
    model_names.append(model_name)
    model_avg_score.append(scores.mean())

In [28]:
df_mode = pd.DataFrame()
df_mode['Model Name'] = model_names
df_mode['Average Score'] = model_avg_score
df_mode

,Model Name,Average Score
0,Logistic Regression,0.829486
1,Random Forest,0.820076
2,Gradient Boosting,0.811714
3,XGBoost,0.785576


In [30]:
for model_name, model in models.items():
  model.fit(x_train_scaled, y_train)
  y_pred = model.predict(x_test_scaled)
  print(f"Classification Report for {model_name}:")
  print(classification_report(y_test, y_pred))

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.83      0.73      0.78        78
           1       0.59      0.71      0.65        42

    accuracy                           0.72       120
   macro avg       0.71      0.72      0.71       120
weighted avg       0.74      0.72      0.73       120

Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.78      0.77      0.77        78
           1       0.58      0.60      0.59        42

    accuracy                           0.71       120
   macro avg       0.68      0.68      0.68       120
weighted avg       0.71      0.71      0.71       120

Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.76      0.82      0.79        78
           1       0.61      0.52      0.56        42

    accuracy                           0.72      

**Hyperparameter tuning of GradientBoos**t


In [32]:
optimizer = GridSearchCV(GradientBoostingClassifier(),{
    'loss':['deviance', 'exponential'],
    'learning_rate':[0.05,0.1,0.5],
    'n_estimators':[50,100,150,200],
    'criterion':['friedman_mse','squared_error'],
    'max_features':['auto','sqrt','log2',None]
})

In [33]:
optimizer.fit(x_train_scaled, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
600 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
480 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'criterion': ['friedman_mse', 'squared_error'],
                         'learning_rate': [0.05, 0.1, 0.5],
                         'loss': ['deviance', 'exponential'],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'n_estimators': [50, 100, 150, 200]})

In [34]:
optimizer.best_params_, optimizer.best_score_

({'criterion': 'friedman_mse',
  'learning_rate': 0.05,
  'loss': 'exponential',
  'max_features': None,
  'n_estimators': 50},
 0.7787280701754387)

In [36]:
best_params = optimizer.best_params_

In [37]:
final_model = GradientBoostingClassifier(criterion=best_params['criterion'],
                                         learning_rate=best_params['learning_rate'],
                                         loss=best_params['loss'],
                                         max_features=best_params['max_features'],
                                         n_estimators=best_params['n_estimators'])

In [38]:
final_model.fit(x_train_scaled, y_train)

GradientBoostingClassifier(learning_rate=0.05, loss='exponential',
                           n_estimators=50)

In [39]:
y_final_model_predict = final_model.predict(x_test_scaled)

In [40]:
print(classification_report(y_test,y_final_model_predict))

              precision    recall  f1-score   support

           0       0.73      0.85      0.79        78
           1       0.60      0.43      0.50        42

    accuracy                           0.70       120
   macro avg       0.67      0.64      0.64       120
weighted avg       0.69      0.70      0.69       120



# Testing the model with new dataset

In [42]:
df_unknown = pd.read_csv('/content/Paitients_Files_Test.csv')
df_unknown.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance
0,ICU200609,1,109,38,18,120,23.1,0.407,26,1
1,ICU200610,1,108,88,19,0,27.1,0.400,24,1
2,ICU200611,6,96,0,0,0,23.7,0.190,28,1
3,ICU200612,1,124,74,36,0,27.8,0.100,30,1
4,ICU200613,7,150,78,29,126,35.2,0.692,54,0


In [43]:
x_unknown = df_unknown.drop('ID',axis='columns')


In [44]:
x_unknown_scaler = scaler.transform(x_unknown)

In [45]:
y_unknown_predict = final_model.predict(x_unknown_scaler)

In [46]:
df_unknown['prediction'] = y_unknown_predict
df_unknown.head()

,ID,PRG,PL,PR,SK,TS,M11,BD2,Age,Insurance,prediction
0,ICU200609,1,109,38,18,120,23.1,0.407,26,1,0
1,ICU200610,1,108,88,19,0,27.1,0.400,24,1,0
2,ICU200611,6,96,0,0,0,23.7,0.190,28,1,0
3,ICU200612,1,124,74,36,0,27.8,0.100,30,1,0
4,ICU200613,7,150,78,29,126,35.2,0.692,54,0,1
